In [215]:
from deap import base, creator, tools  # deap utilities
import random
import numpy as np  # numerical computation
import matplotlib.pyplot as plt   # plotting


In [221]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)



c:\Users\Alienware\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Alienware\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [242]:
IND_SIZE = 3  # chromosome length

toolbox = base.Toolbox()
toolbox.register("attribute", random.randint,0, 20)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list,toolbox.individual)



In [243]:
def evaluate(ind):
    """ Returns the fitness of an individual.
    This is your objective function."""
    F=4*ind[0]+8*ind[1]+12*ind[2]-10*((6*ind[0]+3*ind[1]+12*ind[2])/120)**11
    lista=[]
    lista.append(F)
    return sum(lista),  #  must return a tuple

# Operators
toolbox.register("cross", tools.cxTwoPoint)  
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1) 
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)



In [247]:
def evaluate(ind):
    """ Returns the fitness of an individual.
    This is your objective function."""
    F=abs(ind[0]-ind[1])+abs(ind[0]-ind[2])+abs(ind[1]-ind[2])
    lista=[]
    lista.append(F)
    return sum(lista),  #  must return a tuple

# Operators
toolbox.register("cross", tools.cxTwoPoint)  
toolbox.register("mutate", tools.mutUniformInt, low=0, up=20, indpb=1.0) 
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [225]:
def generationalGA():
    pop = toolbox.population(n=50)  # Registered as a list
    CXPB = 0.5  # Crossover probability
    MUTPB = 0.2  # Mutation probability
    NGEN = 40 # Number of generations
    
    # Evaluate all population first
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    # Generate offspring
    for __ in range(NGEN):
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))  # Generate a deep copy
        
        # Apply crossover
        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.cross(c1, c2)
                # Reset their fitness values
                del c1.fitness.values
                del c2.fitness.values

        # Mutate those selected
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                # Reset fitness values
                del mutant.fitness.values

        # Evaluate non-evaluated individuals in offspring
        invalid_inds = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_inds)
        for ind, fit in zip(invalid_inds, fitnesses):
            ind.fitness.values = fit

        # Replace entire population by the offspring
        pop[:] = offspring
    
    return pop


In [258]:
results = generationalGA()
fitnesses = [i.fitness.getValues()[0] for i in results]

bestpos = fitnesses.index(min(fitnesses))
print("Individual: {0}\n\nFitness: {1}".format(
        results[bestpos], results[bestpos].fitness.getValues()[0]))


Individual: [10 10  9]

Fitness: 2.0
